In [ ]:
import rasterio
import numpy as np
import pylandstats as pls

# Function to read GeoTIFF files
def read_geotiff(file_path):
    with rasterio.open(file_path) as src:
        data = src.read(1)
        transform = src.transform
        crs = src.crs
    return data, transform, crs

# Function to calculate transition matrix
def calculate_transition_matrix(lulc1, lulc2, num_classes):
    transition_matrix = np.zeros((num_classes, num_classes), dtype=int)
    for i in range(num_classes):
        for j in range(num_classes):
            transition_matrix[i, j] = np.sum((lulc1 == i) & (lulc2 == j))
    return transition_matrix

# Function to apply CA-Markov model
def ca_markov(lulc, transition_matrix, num_classes, iterations=1):
    for _ in range(iterations):
        new_lulc = np.copy(lulc)
        for i in range(num_classes):
            for j in range(num_classes):
                mask = lulc == i
                change_prob = transition_matrix[i, j] / np.sum(transition_matrix[i])
                new_lulc[mask] = np.random.choice(num_classes, p=change_prob)
        lulc = new_lulc
    return new_lulc

# Paths to historical LULC GeoTIFF files
lulc_files = ["path/to/lulc_2000.tif", "path/to/lulc_2005.tif", "path/to/lulc_2010.tif"]

# Read historical LULC data
lulc_data = []
transforms = []
crs_list = []
for file in lulc_files:
    data, transform, crs = read_geotiff(file)
    lulc_data.append(data)
    transforms.append(transform)
    crs_list.append(crs)

# Ensure all LULC data have the same shape
shape = lulc_data[0].shape
for data in lulc_data:
    assert data.shape == shape

# Number of LULC classes
num_classes = np.max(lulc_data[0]) + 1

# Calculate transition matrices for each pair of consecutive years
transition_matrices = []
for i in range(len(lulc_data) - 1):
    transition_matrix = calculate_transition_matrix(lulc_data[i], lulc_data[i + 1], num_classes)
    transition_matrices.append(transition_matrix)

# Average transition matrix
average_transition_matrix = np.mean(transition_matrices, axis=0)

# Predict future LULC using CA-Markov
future_lulc = ca_markov(lulc_data[-1], average_transition_matrix, num_classes, iterations=10)

# Save the predicted LULC to a new GeoTIFF file
output_file = "path/to/future_lulc.tif"
with rasterio.open(
    output_file, "w",
    driver="GTiff",
    height=future_lulc.shape[0],
    width=future_lulc.shape[1],
    count=1,
    dtype=future_lulc.dtype,
    crs=crs_list[0],
    transform=transforms[0],
) as dst:
    dst.write(future_lulc, 1)

print("Future LULC prediction saved to", output_file)